# Model Evaluation & Insights

In this notebook, we evaluate the ML models trained in Notebook 3. 
We aim to understand:
- how well each model predicts match outcomes
- which features contribute most to predictions
- differences between linear and nonlinear models

We’ll visualize performance metrics, feature importance, and draw actionable insights. 


In [4]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# loads test set
X_test = pd.read_csv('../data/processed/x_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')

# loads trained models 
logreg_model = joblib.load('../models/logistic_regression.pkl')
rf_model = joblib.load('../models/random_forest.pkl')
